In [ ]:
import cv2
from glob import glob
import pathlib 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten,Dense,Conv3D,MaxPool3D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
disease_path=pathlib.Path("/content/drive/MyDrive/CNN/Medical Leaf")

In [ ]:
a=list(disease_path.glob("Crape Jasmine/*.jpg"))
b=list(disease_path.glob("Drumstick/*.jpg"))
c=list(disease_path.glob("Indian Beech/*.jpg"))
d=list(disease_path.glob("Jamaica Cherry-Gasagase/*.jpg"))
e=list(disease_path.glob("Jasmine/*.jpg"))
f=list(disease_path.glob("Mango/*.jpg"))
g=list(disease_path.glob("Mexican Mint/*.jpg"))
h=list(disease_path.glob("Mint/*.jpg"))
i=list(disease_path.glob("Rose Apple/*.jpg"))
j=list(disease_path.glob("Tulsi/*.jpg"))

In [ ]:
len(a),len(b),len(c),len(d),len(e),len(f),len(g),len(h),len(i),len(j)

(56, 77, 61, 56, 71, 62, 48, 97, 56, 52)

In [ ]:
disease_dict={"Crape Jasmine":a,
              "Drumstick":b,
              "Indian Beech":c,
              "Jamaica Cherry-Gasagase":d,
              "Jasmine":e,
              "Mango":f,
              "Mexican Mint":g,
              "Mint":h,
              "Rose Apple":i,
              "Tulsi":j}
disease_class={"Crape Jasmine":0,
              "Drumstick":1,
              "Indian Beech":2,
              "Jamaica Cherry-Gasagase":3,
              "Jasmine":4,
              "Mango":5,
              "Mexican Mint":6,
              "Mint":7,
              "Rose Apple":8,
              "Tulsi":9}

In [ ]:
x=[]
y=[]

In [ ]:
for i in disease_dict:
  disease_name=i
  disease_path_list=disease_dict[disease_name]
  for path in disease_path_list:
    img=cv2.imread(str(path))
    img=cv2.resize(img,(224,224))
    img=img/255
    x.append(img)
    cls=disease_class[disease_name]
    y.append(cls)

In [ ]:
len(x)

636

In [ ]:
x=np.array(x)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.75,random_state=1)

In [ ]:
len(xtrain),len(ytrain),len(xtest),len(ytest)

(477, 477, 159, 159)

In [ ]:
xtrain.shape

(477, 224, 224, 3)

In [ ]:
xtrain.shape,xtest.shape

((477, 224, 224, 3), (159, 224, 224, 3))

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
print("[INFO] summary for base model...")
print(base_model.summary())

[INFO] summary for base model...
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )             

In [ ]:
from tensorflow.keras.layers import MaxPooling2D
from keras.layers.core import Dropout
from tensorflow.keras.models import Model
# construct the head of the model that will be placed on top of the
# the base model
headModel = base_model.output
headModel = MaxPooling2D(pool_size=(2, 2))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(32, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(10, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=base_model.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in base_model.layers:
	layer.trainable = False

In [ ]:
from tensorflow.keras.optimizers import Adam
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = Adam(lr=1e-4)
model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
#H = model.fit(
	#data_generator.flow(xtrain, ytrain, batch_size=32),
#	steps_per_epoch=len(xtrain) // 32,
	#validation_data=valAug.flow(xtest, ytest),
	#validation_steps=len(xtest) // 32,
#	epochs=5)

model_hist=model.fit(xtrain,ytrain,epochs=5,validation_data=(xtest,ytest),batch_size=180)

[INFO] compiling model...


[INFO] training head...
Epoch 1/5
3/3 [==============================] - 38s 13s/step - loss: 6.8734 - accuracy: 0.1111 - val_loss: 2.7670 - val_accuracy: 0.2642
Epoch 2/5
3/3 [==============================] - 34s 12s/step - loss: 2.2791 - accuracy: 0.2809 - val_loss: 1.3930 - val_accuracy: 0.4654
Epoch 3/5
3/3 [==============================] - 32s 12s/step - loss: 1.5211 - accuracy: 0.4046 - val_loss: 1.3775 - val_accuracy: 0.4969
Epoch 4/5
3/3 [==============================] - 32s 12s/step - loss: 1.4096 - accuracy: 0.4256 - val_loss: 1.2569 - val_accuracy: 0.4843
Epoch 5/5
3/3 [==============================] - 32s 10s/step - loss: 1.2761 - accuracy: 0.4528 - val_loss: 1.1831 - val_accuracy: 0.5094


In [ ]:
model.save("heart.h5")

In [ ]:
from tensorflow.keras.preprocessing import image
# testing the model
def testing_image(image_directory):
    test_image = image.load_img(image_directory, target_size = (224, 224))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    test_image = test_image/255
    result = model.predict(x= test_image)
    print(result)
    if np.argmax(result)  == 0:        
      prediction = 'Crape Jasmine'
    elif np.argmax(result)  == 1:
      prediction = 'Drumstick'
    elif np.argmax(result)==2:
      prediction='Indian Beech'
    elif np.argmax(result)==3:
      prediction='Jamaica Cherry-Gasagase'
    elif np.argmax(result)==4:
      prediction='Jasmine'
    elif np.argmax(result)==5:
      prediction='Mango'
    elif np.argmax(result)==6:
      prediction='Mexican Mint'
    elif np.argmax(result)==7:
      prediction='Mint'
    elif np.argmax(result)==8:
      prediction='Rose Apple'
    
    else:
      prediction = 'Tulsi'
    return prediction

In [ ]:
print(testing_image('/content/drive/MyDrive/CNN/Medical Leaf/Crape Jasmine/TD-S-003.jpg'))

1/1 [==============================] - 1s 883ms/step
[[9.9924707e-01 1.0798183e-06 3.9276044e-04 1.8849468e-08 2.5865601e-04
  3.1695972e-06 5.8626290e-05 5.7555791e-08 3.8671162e-05 4.2580908e-10]]
Crape Jasmine


In [ ]:
print(testing_image('/content/drive/MyDrive/download.jpg'))

1/1 [==============================] - 0s 60ms/step
[[7.9509746e-03 4.6023601e-04 8.5387379e-03 5.9768274e-02 1.5286992e-02
  2.6591545e-01 1.3518229e-02 5.9442276e-01 3.3428550e-02 7.0980535e-04]]
Mint


In [ ]:
print(testing_image("/content/drive/MyDrive/tulsi.jpg"))

1/1 [==============================] - 0s 58ms/step
[[0.00379236 0.00094181 0.06345814 0.02648375 0.01335483 0.4069537
  0.08040792 0.35331008 0.05043002 0.00086737]]
Mango
